# Web Scraping Project

## Graham Website 

In [613]:
# Importing Libraries
import requests
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd 

### Parsing Graham Webpage 

In [614]:
# Parsing URL
main_url = 'https://graham1695.com'
main_html = requests.get(main_url)
main_soup = BeautifulSoup(main_html.content, "html.parser")

### Parsing The Webpage of All The Watches 

In [615]:
for a in main_soup.find_all("a", href=True):
    if "find my graham" in str(a).lower():
        break

In [616]:
url_ext = str(a)[str(a).find("href=")+len("href="):str(a).find(">")].replace('"',"")
url = main_url+url_ext

In [617]:
html = requests.get(url)
soup = BeautifulSoup(html.content, "html.parser")

#### Parsing The Pages

In [618]:
# Rotating through the pages
#pages_element = soup.find_all("span", attrs={"class": "page"})
#pages_url = [watches_url]
#for element in pages_element:
#    a = element.find_all("a", href=True)
#    if a != []:
#        pages_url.append(main_url+str(a[0])[str(a).find('"'):str(a).find(">")-2])

#### Parsing The Products Webpages

In [619]:
# Finding the products urls
#products_url = []
#for page_url in pages_url:
#    page_html = requests.get(page_url)
#    page_soup = BeautifulSoup(page_html.content, "html.parser")
#    products = page_soup.find_all("div", attrs={"class": "grid-product"})
#    for i in range(len(products)):
#        a = str(products[i].find_all("a")[0])
#        products_url.append(main_url+a[a.find("href=")+len("href="):a.find(">")].replace('"',""))
#products_url = list(set(products_url))

#### Parsing Watches Per Collection

In [633]:
collections = soup.find_all("div", attrs={"class":"collection-sidebar__group--1"})[0].find_all("span", attrs={"class":"tag__text"})
collections = [collection.text for collection in collections]
watches_per_collection = {collection: [] for collection in collections}

In [626]:
products_url = []
for collection in collections:
    filter_url = "?filter.p.m.custom.collection="
    collection_url = url+filter_url+collection.replace(" ", "+")
    collection_html = requests.get(collection_url)
    collection_soup = BeautifulSoup(collection_html.content, "html.parser")
    watches_element = collection_soup.find_all("div", attrs={"class": "collection-grid__wrapper"})[0].find_all("div", attrs={"class": "grid-product"})
    watches_per_collection[collection].extend([watches_element[i].find("div", attrs={"class": "grid-product__title"}).text for i in range(len(watches_element))])
    next_page = collection_soup.find_all("span", attrs={"class": "page"})
    products = collection_soup.find_all("div", attrs={"class": "grid-product"})
    for i in range(len(products)):
        a = str(products[i].find_all("a")[0])
        products_url.append(main_url+a[a.find("href=")+len("href="):a.find(">")].replace('"',""))
    if next_page != []:
        for page in next_page[1:]:
            next_page_url = main_url+page.find("a").attrs["href"]
            next_page_html = requests.get(next_page_url)
            next_page_soup = BeautifulSoup(next_page_html.content, "html.parser")
            watches_element = collection_soup.find_all("div", attrs={"class": "collection-grid__wrapper"})[0].find_all("div", attrs={"class": "grid-product"})
            watches_per_collection[collection].extend([watches_element[i].find("div", attrs={"class": "grid-product__title"}).text for i in range(len(watches_element))])
            products = collection_soup.find_all("div", attrs={"class": "grid-product"})
            for i in range(len(products)):
                a = str(products[i].find_all("a")[0])
                products_url.append(main_url+a[a.find("href=")+len("href="):a.find(">")].replace('"',""))
products_url = list(set(products_url))

### Accessing All The Products Webpages

#### Creating A Function For The Required Information

In [645]:
def get_information(dct, soup, url, collections):
    # reference number
    try:
        dct["reference_number"]=soup.find_all(attrs={"class": "product-single__sku"})[0].text.replace("\n","")
    except:
        dct["reference_number"]=""
    # watch url
    dct["watch_URL"]=url
    # brand
    dct["brand"]="Graham"
    # parent model
    for key in collections.keys():
        if data_dict["specific_model"] in collections[key]:
            dct["parent_model"]=key
    # specific model
    dct["specific_model"]=soup.find_all("h1")[0].text.replace("\n","")
    # nickname
    dct["nickname"]=soup.find_all("h1")[0].text.replace("\n","")
    # marketing name
    for h2 in soup.find_all("h2"):
        if "LIMITED EDITION" in h2.text:
            dct["marketing_name"]="Limited Edition"
    # currency and price
    symbol = soup.find_all(attrs={"class": "product__price"})[0].text.replace("\n","")[0]
    price = soup.find_all(attrs={"class": "product__price"})[0].text.replace("\n","")[1:]
    if symbol == "$":
        currency = "USD"
    elif symbol == "€":
        currency = "EUR"
    elif symbol == "C":
        currency = "CHF"
        price = soup.find_all(attrs={"class": "product__price"})[0].text.replace("\n","")[4:]
    else:
        currency = ""
    dct["currency"]=currency
    dct["price"]=price
    # image url
    for img in soup.find_all("img"):
        if dct["specific_model"][0] in str(img):
            break
    img_url = str(img)[str(img).find("data-photoswipe-src=")+len("data-photoswipe-src=")
                 :str(img).find("data-photoswipe-src=")+str(img)[str(img).find("data-photoswipe-src="):].find(" ")
                        ].replace('"',"").replace("//","")
    dct["image_URL"]=img_url
    # made in
    dct["made_in"]="Switzerland"
    # descriprion
    try:
        element = soup.find_all("div", attrs={"class": "filter-result-content"})[0]
    except:
        element = soup.find_all("div", attrs={"class": "rte"})[0]
    description = []
    while True:
        if element.name == "div":
            element = element.find_next()
            continue
        elif element.name == "p":
            description.append(element.text)
            element = element.find_next()
        else:
            break
    dct["description"]=description
    # short descriprion
    for element in soup.find_all("div", attrs={"class": "rte"}):
        if "TECHNICAL SHEET" in str(element):
            a = str(element.find("a"))
            break
    pdf_url = a[a.find("href=")+len("href=")+1: a.find(" title")-len(" title")]
    dct["short_description"]=pdf_url
    # returning the information
    return dct

#### Going Through Each Page to Obtain The Required Information

In [649]:
# Getting the csv template
data_set = pd.read_csv("Graham - Test Scrapes - Sample Data.csv")
data_set = data_set.iloc[: , 1:]
data_set = data_set.drop([0,1])
data_dict = {column : "" for column in data_set.columns}

In [650]:
# Going to the product page
for product_url in products_url:
    product_html = requests.get(product_url)
    product_soup = BeautifulSoup(product_html.content, "html.parser")
    product_dict = get_information(data_dict, product_soup, product_url, watches_per_collection)
    data_set = data_set.append(product_dict, ignore_index = True)
    data_dict = {column : "" for column in data_set.columns}

/var/folders/45/y4_7bsn91fx5ltkk45x0wzpc0000gn/T/ipykernel_50780/2642664573.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_set = data_set.append(product_dict, ignore_index = True)
/var/folders/45/y4_7bsn91fx5ltkk45x0wzpc0000gn/T/ipykernel_50780/2642664573.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_set = data_set.append(product_dict, ignore_index = True)
/var/folders/45/y4_7bsn91fx5ltkk45x0wzpc0000gn/T/ipykernel_50780/2642664573.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_set = data_set.append(product_dict, ignore_index = True)
/var/folders/45/y4_7bsn91fx5ltkk45x0wzpc0000gn/T/ipykernel_50780/2642664573.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pan

/var/folders/45/y4_7bsn91fx5ltkk45x0wzpc0000gn/T/ipykernel_50780/2642664573.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_set = data_set.append(product_dict, ignore_index = True)
/var/folders/45/y4_7bsn91fx5ltkk45x0wzpc0000gn/T/ipykernel_50780/2642664573.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_set = data_set.append(product_dict, ignore_index = True)
/var/folders/45/y4_7bsn91fx5ltkk45x0wzpc0000gn/T/ipykernel_50780/2642664573.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_set = data_set.append(product_dict, ignore_index = True)
/var/folders/45/y4_7bsn91fx5ltkk45x0wzpc0000gn/T/ipykernel_50780/2642664573.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pan

In [652]:
data_set.to_csv("Graham_Watches.csv")